Вітаю! Сьогодні ви data scientist в команді модерації) <br>
Команда маркетингу почала закуповувати більше трафіку і навантаження на фото-модераторів росте з кожним днем, але команда модерації не встигає так швидко розшируватись, оскільки пошук і навчання модераторів займає час.

Ваша задача автоматизувати частину роботи фото-модераторів, щоб їм не доводилось працювати по ночам і в стресі.
Ви вирішили автоматизувати одну з найпоширеніших причин відхилення фото – групові фото.
Проблема полягає в тому, що не всі фото, на яких є два і більше облич відхиляються модераторами.
Отже наша задача натренувати модель машинного навчання, яка буде відрізняти "хороші" фото з декількома обличчями від "поганих".

У відповідь чекаю цей заповнений ноутбук <br>
По необхідності пишіть в тг https://t.me/crodetsya

Успіхів!

# Імпорт необхідних бібліотек

In [15]:
!pip install tqdm==4.61.0

^C


In [ ]:
import pandas as pd
import json
from tqdm.notebook import tqdm
import traceback

# Дані


Дані лежать в двох файлах **approved_100k_annotations.csv** та **declined_100k_annotations.csv** <br>
І представляють дві групи по 100 тисяч відхилених та апрувнутих на ручній модерації фото з деякими метаданими, отриманими з моделькі face detection

Кожен рядок файлу має структуру *filename, annotations*, де *annotations* –  json об'єкт і має наступну структуру:

"face_annotations" - це лист, довжина якого дорівнює кількості знайдених на фото облич 

    кожне обличчя має "bounding_poly" який містить в собі координати прямокутника навкола знайденого обличчя у вигляді листа 
    "vertices" елементами якого є словники виду {"x": int, "y": int}

    кожне обличчя має "landmarks" – лист який містить в собі 5 точок, які позначають положення правого/лівого ока, кончика носа, правого/лівого куточка рота у вигляді словника {"type": int(тип точки), "position": {"x": int, "y": int} }

    кожне обличчя має "detection_confidence" – впевненість моделі face detection, що в цьому прямокутнику є обличчя


    "safe_serve_annotation" нам не потрібно

## Приклад аnnotations одного фото

<details>
<summary style="cursor: pointer;
    border: 1px solid gray;
    padding: 5px;
    background: #fff7af;">Клікніть на цей текст, щоб подивитись приклад annotations в розгорнутому вигляді</summary>
<pre>
{
    "face_annotations": [
        {
            "bounding_poly": {
                "vertices": [
                    {
                        "x": 44,
                        "y": 24
                    },
                    {
                        "x": 275,
                        "y": 24
                    },
                    {
                        "x": 275,
                        "y": 357
                    },
                    {
                        "x": 44,
                        "y": 357
                    }
                ]
            },
            "landmarks": [
                {
                    "type": 1,
                    "position": {
                        "x": 103,
                        "y": 167
                    }
                },
                {
                    "type": 2,
                    "position": {
                        "x": 216,
                        "y": 161
                    }
                },
                {
                    "type": 3,
                    "position": {
                        "x": 165,
                        "y": 233
                    }
                },
                {
                    "type": 4,
                    "position": {
                        "x": 122,
                        "y": 282
                    }
                },
                {
                    "type": 5,
                    "position": {
                        "x": 215,
                        "y": 277
                    }
                }
            ],
            "detection_confidence": 0.99813986
        },
        {
            "bounding_poly": {
                "vertices": [
                    {
                        "x": 364,
                        "y": 152
                    },
                    {
                        "x": 392,
                        "y": 152
                    },
                    {
                        "x": 392,
                        "y": 192
                    },
                    {
                        "x": 364,
                        "y": 192
                    }
                ]
            },
            "landmarks": [
                {
                    "type": 1,
                    "position": {
                        "x": 385,
                        "y": 163
                    }
                },
                {
                    "type": 2,
                    "position": {
                        "x": 387,
                        "y": 163
                    }
                },
                {
                    "type": 3,
                    "position": {
                        "x": 392,
                        "y": 172
                    }
                },
                {
                    "type": 4,
                    "position": {
                        "x": 384,
                        "y": 181
                    }
                },
                {
                    "type": 5,
                    "position": {
                        "x": 386,
                        "y": 181
                    }
                }
            ],
            "detection_confidence": 0.9535075
        }
    ],
    "safe_serve_annotation": {
        "nsfw_confidence": 5.9604645e-07
    }
}
    </pre>

</details>

# Парсинг даних

## Завдання 1 
Дописати функцію parse_csv яка парсить csv файл заданої вище структури і повертає pandas.DataFrame з наступними колонками: <br>
```json
filename,
Для кожного обличчя* i[0,5){
    face{i}_score,**
    
    face{i}_bbox_x1,
    face{i}_bbox_y1,
    face{i}_bbox_x2,
    face{i}_bbox_y2,
    
    Для кожної лендмарки j[1,6){
        face{i}_landmark_x{j},
        face{i}_landmark_y{j}
    }
}
```
\* облич максимум 5, навіть якщо є дані про більше, то їх не враховуємо <br>
\*\* bbox – bounding box, прямокутник навколо обличчя

**Алгоритм функції parse_csv:**
1. відкрити файл, створити пустий список в який ми будемо додавати рядки нашого датафрейму у вигляді словників
2. розбиту кожен рядок на filename та annotations
3. перевести annotations в словник (бібілотека json)
4. для кожного рядка файлу формуємо словник {"назва колонки: значення"}
5. стоврений словник додаємо до датафрейму(який поки що просто список)
6. зі списка створюємо pandas.DataFrame, вказуючи колонки

Створючи спочатку список словників, ми економимо дуже велику кількість часу, порівняно з тим, щоб одразу створювати pandas.DataFrame і додавати в нього дані рядками

Дані містять в собі помилки, тож ми відловлюємо ексепшени і просто пропускаємо цей рядок

In [ ]:
columns = ['filename'] # колонки результуючого датасету

for i in range(0,5):
    columns.append(f"face{i}_score")
    
    for j in range(1,3):
        columns.append(f'face{i}_bbox_x{j}')
        columns.append(f'face{i}_bbox_y{j}')
        
    for k in range(1,6):
        columns.append(f"face{i}_landmark_x{k}")
        columns.append(f"face{i}_landmark_y{k}")
columns

['filename',
 'face0_score',
 'face0_bbox_x1',
 'face0_bbox_y1',
 'face0_bbox_x2',
 'face0_bbox_y2',
 'face0_landmark_x1',
 'face0_landmark_y1',
 'face0_landmark_x2',
 'face0_landmark_y2',
 'face0_landmark_x3',
 'face0_landmark_y3',
 'face0_landmark_x4',
 'face0_landmark_y4',
 'face0_landmark_x5',
 'face0_landmark_y5',
 'face1_score',
 'face1_bbox_x1',
 'face1_bbox_y1',
 'face1_bbox_x2',
 'face1_bbox_y2',
 'face1_landmark_x1',
 'face1_landmark_y1',
 'face1_landmark_x2',
 'face1_landmark_y2',
 'face1_landmark_x3',
 'face1_landmark_y3',
 'face1_landmark_x4',
 'face1_landmark_y4',
 'face1_landmark_x5',
 'face1_landmark_y5',
 'face2_score',
 'face2_bbox_x1',
 'face2_bbox_y1',
 'face2_bbox_x2',
 'face2_bbox_y2',
 'face2_landmark_x1',
 'face2_landmark_y1',
 'face2_landmark_x2',
 'face2_landmark_y2',
 'face2_landmark_x3',
 'face2_landmark_y3',
 'face2_landmark_x4',
 'face2_landmark_y4',
 'face2_landmark_x5',
 'face2_landmark_y5',
 'face3_score',
 'face3_bbox_x1',
 'face3_bbox_y1',
 'face3_bbo

<details>
<summary style="cursor: pointer;
    border: 1px solid gray;
    padding: 5px;
    background: #fff7af;">Підказки для parse_csv(не клікайте одразу, спробуйте самі спочатку)</summary>
<pre>
2. використати split з аргументом maxsplit=1
3. використати json.loads
</pre>
</details>

In [ ]:
def parse_csv(csv_name: str, debug=False):
    with open(csv_name, 'r') as file:
        errors_count = 0 # кількість помилок при обробці рядка
        parsed_df = [] # результуючий датафрейм

        for line_num, line in tqdm(enumerate(file.readlines()), total=100000):
            if line_num == 0: # перший рядок – заголовок
                continue

            filename, json_annotations = line.split(',', maxsplit=1)
            annotations = json.loads(json_annotations)

            try: 
                
                row_dict = {"filename": filename}

                for i, face in enumerate(annotations['face_annotations'][0:5]):
                    row_dict[f'face{i}_score'] = face['detection_confidence']

                    bbox = face['bounding_poly']['vertices']

                    row_dict[f'face{i}_bbox_x1'] = bbox[0]['x']
                    row_dict[f'face{i}_bbox_y1'] = bbox[0]['y']
                    row_dict[f'face{i}_bbox_x2'] = bbox[2]['x'] # цей і наступний рядок – не помилка,
                    row_dict[f'face{i}_bbox_y2'] = bbox[3]['y'] # бо так як у нас прямокутник, то координати точок повторюються

                    for a in face['landmarks']:
                        l_type, l_point = a.values() # тип лендмарки(від 1 до 6 не включно), point = {'x': int, 'y': int}
                        row_dict[f'face{i}_landmark_x{l_type}'] = l_point['x']
                        row_dict[f'face{i}_landmark_y{l_type}'] = l_point['y']

                parsed_df.append(row_dict)
                
            except Exception as e:
                if debug:
                    traceback.print_exc()
                    errors_count += 1
                    print(f"Percent of errors: {errors_count/line_num}, errors_count:{errors_count}")
                continue
                
    parsed_df = pd.DataFrame.from_dict(parsed_df)           
    return parsed_df


## Перевірка функції parse_csv
Наступні дві клітинки змінювати не потрібно, треба лише запустити їх по черзі <br>
Якщо функція parse_csv написана правильно, то після завершення виконання побачите "Nice 🙂"

In [ ]:
def test_parse_csv(f):
    print('Parsing test dataframe...')
    test_df = parse_csv('test_annotations.csv', debug=True)
    test_parsed_df = pd.read_csv('test_parsed_annotations.csv')
    
    print('Comapring test dataframe...')
    for i, row in tqdm(test_df.iterrows(), total=len(test_df)):
        p_row = test_parsed_df.loc[test_parsed_df.filename == row.filename, :]
            
        dict1 = row.to_dict()
        dict2 = p_row.squeeze(axis=0).to_dict()
        for key, value in dict1.items():
            if value != dict2[key]:
                if pd.isna(value) and pd.isna(dict2[key]):
                    continue
                print(f"Filename {row.filename}")
                print(f"Ви неправильно рахуєте {key}")
                print(f"Ваше значення {value}, не співпадає з тестовим {dict2[key]}")
                print(" ")
                return
            
    print('Nice 🙂')

In [ ]:
test_parse_csv(parse_csv)

Parsing test dataframe...


  0%|          | 0/100000 [00:00<?, ?it/s]

Percent of errors: 0.07692307692307693, errors_count:1
Percent of errors: 0.07407407407407407, errors_count:2
Percent of errors: 0.03125, errors_count:3
Percent of errors: 0.04, errors_count:4
Percent of errors: 0.04201680672268908, errors_count:5
Percent of errors: 0.046511627906976744, errors_count:6
Percent of errors: 0.0472972972972973, errors_count:7
Percent of errors: 0.0380952380952381, errors_count:8
Percent of errors: 0.032490974729241874, errors_count:9
Percent of errors: 0.0353356890459364, errors_count:10
Percent of errors: 0.025821596244131457, errors_count:11
Percent of errors: 0.025210084033613446, errors_count:12
Percent of errors: 0.026, errors_count:13
Percent of errors: 0.02676864244741874, errors_count:14
Percent of errors: 0.024875621890547265, errors_count:15
Percent of errors: 0.02576489533011272, errors_count:16
Percent of errors: 0.023876404494382022, errors_count:17
Percent of errors: 0.022388059701492536, errors_count:18
Percent of errors: 0.02115812917594654

Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Temp\ipykernel_9560\1820215816.py", line 22, in parse_csv
    row_dict[f'face{i}_bbox_x1'] = bbox[0]['x']
KeyError: 'x'
Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Temp\ipykernel_9560\1820215816.py", line 29, in parse_csv
    row_dict[f'face{i}_landmark_x{l_type}'] = l_point['x']
KeyError: 'x'
Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Temp\ipykernel_9560\1820215816.py", line 22, in parse_csv
    row_dict[f'face{i}_bbox_x1'] = bbox[0]['x']
KeyError: 'x'
Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Temp\ipykernel_9560\1820215816.py", line 29, in parse_csv
    row_dict[f'face{i}_landmark_x{l_type}'] = l_point['x']
KeyError: 'x'
Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Temp\ipykernel_9560\1820215816.py", line 22, in parse_csv
    row_dict[f'face{i}_bbox_x1'] = bbox[0]['x']
KeyError: 'x'
Traceback (most recent 

  0%|          | 0/1970 [00:00<?, ?it/s]

Nice 🙂


# Завдання 2
Розпарсити два csv файла, додати колонку target та об'єднати їх з датафреймом images(filename, height, width, face_count) в один pandas.DataFrame <br>

В кінці у вас повинен бути один датафрейм images з усімами фічами з анотацій + target + height, width, face_count 

In [ ]:
approved_df = parse_csv('approved_100k_annotations.csv')

  0%|          | 0/100000 [00:00<?, ?it/s]

In [ ]:
declined_df = parse_csv('declined_100k_annotations.csv')

  0%|          | 0/100000 [00:00<?, ?it/s]

In [ ]:
approved_df['target'] = 1
declined_df['target'] = 0

In [ ]:
concatenated_df = pd.concat([approved_df, declined_df])

In [ ]:
images = pd.read_csv('images_200k.csv', index_col=0)
images

,filename,face_count,width,height
0,5c654836493373c6c4daeab38dc7823f5ee6b9b1,4,1500,2000
1,45ba2a15fd43ad7d31a541551dd3c378fc0e727a,2,1080,1088
2,cc843d592922e1398d45489f992de1f80c6f29a8,2,1502,2000
3,2c1451615b078ebced000c4b3936772444977eb8,2,1500,2000
4,13d0af9e4f9ad9ce168aeb8c24940923bf51e260,2,640,640
...,...,...,...,...
99995,4ee654a06322eaf5f0dc9b2cab27abfc3f90003b,2,720,1280
99996,e2cd152b38ea0c973cf536d89e125d5eab4fd635,10,720,896
99997,1eeb1f66cb7cf19bb77798a7b5018c1935f484ce,4,750,1000
99998,1ac5c491848ab4237e7c99af1b92dca8f05f223a,2,750,1000


Злиття images та concatenated_df повинно бути аналогічно inner join, бо нам не потрібні рядки без анотацій

In [ ]:
images = pd.merge(concatenated_df, images, on='filename',  how='left')

In [ ]:
images

,filename,face0_score,face0_bbox_x1,face0_bbox_y1,face0_bbox_x2,face0_bbox_y2,face0_landmark_x1,face0_landmark_y1,face0_landmark_x2,face0_landmark_y2,...,face4_landmark_x3,face4_landmark_y3,face4_landmark_x4,face4_landmark_y4,face4_landmark_x5,face4_landmark_y5,target,face_count,width,height
0,0025787b3cca65a28374828134b3a81d58340ccf,0.998140,44,24,275,357,103,167,216,161,...,NaN,NaN,NaN,NaN,NaN,NaN,1,2,415,720
1,009a7343a079e2f97ecf92400808678bf87afb98,0.998625,777,570,1331,1320,908,874,1166,826,...,NaN,NaN,NaN,NaN,NaN,NaN,1,3,1500,2000
2,00fd9eba98db9e9afd01508b36eb5e2a56cad0fd,0.999161,526,422,656,616,611,498,636,496,...,NaN,NaN,NaN,NaN,NaN,NaN,1,2,1506,2000
3,0118643c6be1503e83c8cc7254d2ed77a7517224,0.999534,598,801,960,1317,688,978,860,993,...,1205.0,1299.0,1195.0,1311.0,1209.0,1310.0,1,6,1500,2000
4,02c066ebbc41ad55c4a87fbe61ad87d455c80839,0.999194,24,390,82,483,68,425,69,423,...,229.0,430.0,219.0,441.0,233.0,442.0,1,7,900,1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196238,fe9d4390382ce911f7b052cf0a8baf0e2f9b0150,0.999922,-22,232,255,426,76,390,83,297,...,NaN,NaN,NaN,NaN,NaN,NaN,0,4,675,1200
196239,ff078a104e70b3a5759686cf0a6903929e47c2fb,0.998174,296,919,392,1038,319,970,363,965,...,NaN,NaN,NaN,NaN,NaN,NaN,0,3,1500,2000
196240,ff2b5389e423e73348d174a750074f2b12cf7b16,0.998405,239,222,480,520,336,338,443,341,...,NaN,NaN,NaN,NaN,NaN,NaN,0,2,2000,920
196241,ff81899f0efd2808c6aa970ed4995945223d676b,0.999886,-128,50,855,839,441,318,587,604,...,NaN,NaN,NaN,NaN,NaN,NaN,0,3,1125,2000


# Завдання 3
0. (Необов'язковий крок) Придумати і додати додаткові фічі в датафрейм. Наприклад, площа зображення, площа кожного обличчя, відношення між ними, всілякі відстані. Або якусь додаткову обробку даних, яка покращить результат.
1. Розбити датафрейм на train/test/validation
2. Навчити catboost модель, або будь-яку модель на вибір
3. Порахувати auc, зробити classification_report на валідаційній вибірці

In [ ]:
!pip install catboost==0.26

^C


In [18]:
from catboost import CatBoostClassifier as cbc
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(images.drop(['filename', 'target'], axis=1),
                                                    images['target'],
                                                    test_size=0.1, 
                                                    random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                  y_train, 
                                                  test_size=0.05, 
                                                  random_state=1) 

In [20]:
len(X_train), len(X_test), len(X_val)

(167787, 19625, 8831)

In [21]:
drop_columns = ['filename', 'target'] # колонки які не повинні брати участь в тесті, та при оцінці метрик

https://catboost.ai/en/docs/references/training-parameters/

Параметри моделі можете змінювати, щоб досягти кращого результату

In [22]:
params = {
          'loss_function':'Logloss', # objective function
          'eval_metric':'AUC', # metric
          'custom_loss':['AUC', 'Accuracy', 'Precision'],
          'verbose': 200, # output to stdout info about training process every 200 iterations
          'random_seed': 42,
          'iterations': 2500
         }
model = cbc(**params)

In [23]:
model.fit(X_train,
          y_train,
          eval_set=(X_val, y_val),
          use_best_model=True, 
          plot=True 
         )

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\catboost\core.py:1133: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, pairs, weight, group_id, group_weight, subgroup_id, pairs_weight, baseline, feature_names, thread_count)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.075391
0:	test: 0.8377208	best: 0.8377208 (0)	total: 234ms	remaining: 9m 45s
200:	test: 0.9094240	best: 0.9094240 (200)	total: 9.87s	remaining: 1m 52s
400:	test: 0.9193599	best: 0.9193599 (400)	total: 18.9s	remaining: 1m 38s
600:	test: 0.9233592	best: 0.9233592 (600)	total: 28.1s	remaining: 1m 28s
800:	test: 0.9253506	best: 0.9253506 (800)	total: 37.2s	remaining: 1m 19s
1000:	test: 0.9267656	best: 0.9268004 (991)	total: 46.3s	remaining: 1m 9s
1200:	test: 0.9276846	best: 0.9276913 (1198)	total: 54.6s	remaining: 59.1s
1400:	test: 0.9283344	best: 0.9283344 (1400)	total: 1m 3s	remaining: 49.5s
1600:	test: 0.9288620	best: 0.9288667 (1590)	total: 1m 11s	remaining: 40.2s
1800:	test: 0.9293840	best: 0.9294213 (1786)	total: 1m 20s	remaining: 31.1s
2000:	test: 0.9298244	best: 0.9298244 (2000)	total: 1m 28s	remaining: 22.1s
2200:	test: 0.9300525	best: 0.9300525 (2200)	total: 1m 37s	remaining: 13.2s
2400:	test: 0.9303124	best: 0.9303124 (2400)	total: 1m 45s	remaining: 4.36s


https://catboost.ai/docs/features/feature-importances-calculation

In [24]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,face1_score,14.277898
1,width,7.988932
2,face0_score,6.722303
3,height,6.462403
4,face1_bbox_y1,6.301038
...,...,...
73,face4_landmark_x1,0.039957
74,face4_landmark_y3,0.037238
75,face4_landmark_y2,0.033843
76,face4_landmark_x2,0.029512


In [25]:
from sklearn.metrics import classification_report, roc_auc_score

In [26]:
roc_auc_score(y_test,
              model.predict_proba(X_test)[:, 1]
             )

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\catboost\core.py:1133: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, pairs, weight, group_id, group_weight, subgroup_id, pairs_weight, baseline, feature_names, thread_count)


0.9342503974966674

In [27]:
print(classification_report(y_test,
                            model.predict(X_test)     
                           ))

              precision    recall  f1-score   support

           0       0.87      0.86      0.87      9821
           1       0.86      0.88      0.87      9804

    accuracy                           0.87     19625
   macro avg       0.87      0.87      0.87     19625
weighted avg       0.87      0.87      0.87     19625



c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\catboost\core.py:1133: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, pairs, weight, group_id, group_weight, subgroup_id, pairs_weight, baseline, feature_names, thread_count)
